In [4]:
import pandas as pd
import plotly.express as px

# Read the CSV file
df = pd.read_csv('options_positions.csv')

# Data cleaning and type conversion
df['Quantity'] = df['Quantity'].apply(lambda x: float(str(x).replace(',', '')))
df['Notl ($)'] = df['Notl ($)'].apply(lambda x: float(str(x).replace('$', '').replace(',', '')))
df['Expiry Date'] = pd.to_datetime(df['Expiry Date'], format='%d-%b-%y')

# Filter to include only 'Active' status
df_active = df[df['Status'] == 'Active']

# Create a new column for trade type
df_active['Trade Type'] = df_active['Direction'] + ' ' + df_active['Call/Put']

# Sort by 'Expiry Date'
df_active_sorted = df_active.sort_values(by='Expiry Date')

# Assuming 'Strike' is a column in 'df_active' DataFrame
grouped = df_active_sorted.groupby(['Ticker', 'Trade Type', 'Expiry Date', 'Strike'])['Notl ($)'].sum().reset_index()

# Create the plot with facets stacked vertically
fig = px.bar(grouped, x='Ticker', y='Notl ($)', color='Trade Type',
             pattern_shape='Trade Type', facet_row='Expiry Date',
             hover_data=['Strike'],  # Include 'Strike' in the hover data
             category_orders={"Trade Type": ["Buy Call", "Buy Put", "Sell Call", "Sell Put"],
                              "Expiry Date": sorted(grouped['Expiry Date'].unique())},
             labels={'Notl ($)': 'Notional ($)', 'Expiry Date': 'Expiry'})

# Update layout for readability
fig.update_layout(
    title='Notional ($) Size of Active Trades by Trade Type and Expiry Date',
    xaxis_title='Ticker',
    yaxis_title='Notional ($)',
    legend_title='Trade Type',
    barmode='group',
    height=300 * len(grouped['Expiry Date'].unique())  # Adjust height based on number of expiration dates
)

# Set the tickers as x-axis titles for each subplot
for d in fig.layout:
    if d.startswith('xaxis') and d != 'xaxis':
        fig.layout[d].title.text = 'Ticker'

# Show the figure
fig.show()

C:\Users\apeir\AppData\Local\Temp\ipykernel_9592\774633942.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
import plotly.io as pio
pio.renderers.default = 'notebook'  # or 'jupyterlab' if you are using JupyterLab

In [17]:
import pandas as pd
import plotly.express as px

from plotly.subplots import make_subplots
import plotly.graph_objects as go


# Read the CSV file
df = pd.read_csv('options_positions.csv')

# Data cleaning and type conversion
df['Quantity'] = df['Quantity'].apply(lambda x: float(str(x).replace(',', '')))
df['Notl ($)'] = df['Notl ($)'].apply(lambda x: float(str(x).replace('$', '').replace(',', '')))
df['Expiry Date'] = pd.to_datetime(df['Expiry Date'], format='%d-%b-%y')

# Filter to include only 'Active' status
df_active = df[df['Status'] == 'Active']

# Create a new column for trade type
df_active['Trade Type'] = df_active['Direction'] + ' ' + df_active['Call/Put']

# Sort by 'Expiry Date'
df_active_sorted = df_active.sort_values(by='Expiry Date', ascending=False)

# Assuming 'Strike' is a column in 'df_active' DataFrame
grouped = df_active_sorted.groupby(['Ticker', 'Trade Type', 'Expiry Date', 'Strike'])['Notl ($)'].sum().reset_index()

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Ensure the data is sorted by 'Expiry Date' and then by 'Ticker'
grouped_sorted = grouped.sort_values(by=['Expiry Date', 'Ticker'])

# Modify 'Notl ($)' for 'Sell Call' and 'Buy Put'
grouped['Notl ($)'] = grouped.apply(lambda row: -row['Notl ($)'] if row['Trade Type'] in ['Sell Call', 'Buy Put'] else row['Notl ($)'], axis=1)

# Create a subplot for each unique 'Expiry Date'
unique_dates = grouped['Expiry Date'].dt.date.unique()
fig = make_subplots(rows=len(unique_dates), cols=1, shared_xaxes=True,
                    subplot_titles=[date.strftime('%Y-%m-%d') for date in unique_dates],
                    vertical_spacing=0.02)

# Iterate over each expiry date to create each subplot
for i, date in enumerate(unique_dates, start=1):
    # Filter the data for the specific date
    date_data = grouped[grouped['Expiry Date'].dt.date == date]
    
    # Append trace for each ticker within the filtered data
    for ticker in date_data['Ticker'].unique():
        ticker_data = date_data[date_data['Ticker'] == ticker]
        fig.append_trace(go.Bar(
            x=ticker_data['Ticker'],
            y=ticker_data['Notl ($)'],
            text=ticker_data['Strike'].astype(str),
            hoverinfo='text',
            hovertext=ticker_data.apply(lambda row: f"{row['Ticker']}, Notional: {row['Notl ($)']}, Strike: {row['Strike']}, Expiry: {row['Expiry Date'].date()}", axis=1),
            name=ticker  # Ticker name as legend entry
        ), row=i, col=1)

# Update layout for readability
fig.update_layout(
    title='Notional ($) Size of Active Trades by Ticker and Expiry Date',
    barmode='group',
    height=300 * len(unique_dates),  # Adjust height based on number of expiration dates
)

# Update x-axis and y-axis for each plot
for i in range(1, len(unique_dates) + 1):
    fig['layout'][f'xaxis{i}']['title'] = 'Ticker'
    fig['layout'][f'xaxis{i}']['tickmode'] = 'array'
    fig['layout'][f'xaxis{i}']['tickvals'] = grouped_sorted['Ticker'].unique()
    fig['layout'][f'xaxis{i}']['ticktext'] = grouped_sorted['Ticker'].unique()
    fig['layout'][f'yaxis{i}']['title'] = 'Notional ($)'

# Show the figure
fig.show()

C:\Users\apeir\AppData\Local\Temp\ipykernel_9592\101802251.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

